# Models Creation

In [1]:
import cv2
import numpy as np
import os
import sys
import pandas as pd

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.5.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/mateoherrera/anaconda3/envs/proyecto_i3d/lib/python3.8/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras-nightly, keras-preprocessing, numpy, opt-einsum, protobuf, six, tensorboard, tensorflow-estimator, termcolor, typing-extensions, wheel, wrapt
Required-by: 


In [ ]:
import tensorflow as tf

In [ ]:
# Agrega el path al directorio raíz del proyecto
sys.path.append(os.path.abspath(".."))
from models import i3d

AttributeError: module 'tensorflow' has no attribute 'Tensor'

In [ ]:
train_df = pd.read_csv("../data/ml/train.csv")
test_df = pd.read_csv("../data/ml/test.csv")
val_df = pd.read_csv("../data/ml/valid.csv")

In [ ]:
def load_video_frames(video_path, start_frame, end_frame, num_frames=32, frame_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: No se pudo abrir el video {video_path}")
        return None

    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Ajustar el rango si es necesario
    start_frame = int(start_frame) if not np.isnan(start_frame) else 0
    end_frame = int(end_frame) if not np.isnan(end_frame) else total_frames

    step = max(1, (end_frame - start_frame) // num_frames)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    for _ in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, frame_size)  # Redimensionar
        frame = frame / 255.0  # Normalizar a [0,1]
        frames.append(frame)

        for _ in range(step - 1):  # Saltar frames para ajustarse al total
            cap.read()

    cap.release()

    # Si el video es muy corto, rellenar con frames vacíos
    while len(frames) < num_frames:
        frames.append(np.zeros((*frame_size, 3)))

    return np.array(frames, dtype=np.float32)  # Shape: (32, 224, 224, 3)


In [ ]:
def dataframe_to_tf_dataset(df, batch_size=8, shuffle=True):
    file_paths = df["Path"].values
    start_frames = df["Start_Frame"].values
    end_frames = df["End_Frame"].values
    labels = df["Evento"].apply(lambda x: 0 if x == "Normal" else 1).values  # Crimen=1, Normal=0

    def generator():
        for i in range(len(file_paths)):
            video_tensor = load_video_frames(file_paths[i], start_frames[i], end_frames[i])
            yield video_tensor, labels[i]

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(32, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32),
        ),
    )

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df))

    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


## I3D Model

## I3D + ConvLSTM2D